In [1]:
import cv2
import numpy as np
from dataloader import Trial
# import matplotlib.pyplot as plt

In [2]:
video_path = r'..\data\TH8\TH8ST17A3D250TS0_2.mov'

In [8]:
recording = Trial(video_path)
# other = recording.to_gray()
recording.to_gray(inplace=True)
# recording.change_contrast(alpha=2, inplace=True)
# recording.gauss_blur(ksize=(5,5), sigmaX=1, inplace=True, sigmaY=1)
# recording.gauss_blur(ksize=(5,5), sigmaX=1, inplace=True, sigmaY=1)
# recording.sharpen(inplace=True)
# recording.gauss_blur(ksize=(3,3), sigmaX=1, inplace=True, sigmaY=1)
# recording.gauss_blur(ksize=(3,3), sigmaX=1, inplace=True, sigmaY=1)
# recording.gauss_blur(ksize=(3,3), sigmaX=1, inplace=True, sigmaY=1)

# recording.dilate(inplace=True, ksize=(3,3))
# recording.erode(inplace=True, ksize=(3,3))

In [9]:
recording.edges(inplace=True, thresh1=20, thresh2=100)

In [7]:
recording.play_video()

In [ ]:
recording.show(recording[0])

In [ ]:
recording.color_reduce()

In [ ]:
# recording.show(last_state[0])
recording.undo()

### Contours
This was me trying to figure out contours and whatnot

In [ ]:
img = recording[0].copy()
c_min = 10
c_delta = 20
c_max = c_min + c_delta

ahhh = 190
lower_color = np.array([c_min, 100, 0])
upper_color = np.array([c_max, ahhh, ahhh])
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, lower_color, upper_color)
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


largest = max(contours, key=cv2.contourArea)
(x, y), radius = cv2.minEnclosingCircle(largest)
print(x)

img = cv2.circle(img, (int(x), int(y)), 10, (0, 255, 0), 2)

cv2.imshow("mask", mask)
# cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
recording.show(recording[0])

In [ ]:
out = recording.edges(thresh1=20, thresh2=50)
# out.dilate(inplace=True, ksize=(5,5))
# out.erode(inplace=True, ksize=(5,5))
# out = out.create_overlay(other=Trial(video_path))
# out = out.create_overlay(other=Trial(video_path))
out.play_video()

In [ ]:
out.play_video()

In [ ]:
recording.play_video()

In [ ]:
recording.edges(thresh1=20, thresh2=120, inplace=True)

In [ ]:
overlayed = recording.create_overlay()
overlayed.play_video()

In [ ]:
ret, labels = cv2.connectedComponents(recording[0])
normalized_labels = labels / ret
label_hue = np.uint8(179 * normalized_labels)
blank_ch = np.uint8(255*np.ones_like(label_hue))
labeled_img_hsv = cv2.merge([label_hue, blank_ch, blank_ch])
print(labeled_img_hsv.shape)
labeled_img_bgr = cv2.cvtColor(labeled_img_hsv, cv2.COLOR_HSV2BGR)
labeled_img_bgr[label_hue==0] = 0
cv2.imshow("Test", labeled_img_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
# recording[0]
# tempo
# ret

## Making the average image

In [12]:
for frame in recording[:]:
    negative = cv2.bitwise_not(frame)
    cv2.imshow("Negative", negative)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    break
